# 0.0 Imports

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

# 1.0 Selenium Scrapping

## 1.1 Scrapping Fifa Ratings Page

In [3]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get( "https://www.fifa.com/fifa-world-ranking/men?dateId=id13792" )

driver.set_window_size(1024, 600)
driver.maximize_window()

time.sleep(2)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "onetrust-reject-all-handler"))
    )
    element.click()
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "table_rankingTable__7gmVl"))
    )
    fifa_table_page1 = element.text
     
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[text()='2']"))
    )
    
    time.sleep(3)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(3)
    
    element.click()   

    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, "table_rankingTable__7gmVl"))
    )
    
    fifa_table_page2 = element.text
    
    driver.quit()  
except:
    driver.quit()

## 1.2 Scrapping Elo Ratings Page

In [4]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get( "https://www.eloratings.net/" )

driver.set_window_size(2000, 1000)
driver.maximize_window()

time.sleep(3)
search = driver.find_element_by_id("maintable_World")
elo_table = search.text

driver.quit()

### Old code

In [132]:
#PEGANDO AS TABELAS NO SITE DA FIFA
#FUNCIONA TBM, MAS O ANTERIOR EH MAIS ORGANIZADO
#ESSE FOI O PRIMEIRO USADO PARA TEST

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.set_window_size(2000, 1000)
driver.maximize_window()

driver.get( "https://www.fifa.com/fifa-world-ranking/men?dateId=id13792" )

time.sleep(2)
link = driver.find_element_by_id("onetrust-reject-all-handler")
link.click()

search = driver.find_element(By.CLASS_NAME, "table_rankingTable__7gmVl")
fifa_table = search.text

link2 = driver.find_element_by_xpath("//div[text()='2']")
time.sleep(3)
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link2)
time.sleep(3)
link2.click()

search2 = driver.find_element(By.CLASS_NAME, "table_rankingTable__7gmVl")
fifa_table2 = search2.text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="onetrust-reject-all-handler"]"}
  (Session info: chrome=107.0.5304.107)


# 3.0 Data wrangling

In [58]:
def soup(table):
    
    #cleaned_soup = BeautifulSoup( table, 'html.parser')
    cleaned_soup = str(table)
    cleaned_soup = cleaned_soup.split('\n')
    
    return cleaned_soup

## 3.1 Fifa Page Wrangling

In [59]:
fifa_ratings_page1 = soup(fifa_table_page1)
fifa_ratings_page2 = soup(fifa_table_page2)

In [60]:
del fifa_ratings_page2[:6]

In [61]:
fifa_ratings = fifa_ratings_page1
fifa_ratings.extend(fifa_ratings_page2)

In [62]:
fifa_ratings

['RK',
 'Team',
 'Total Points',
 'Previous Points',
 '+/-',
 'Positions',
 '1',
 'Brazil',
 '1841.3',
 '1837.56 3.74',
 '2',
 'Belgium',
 '1816.71',
 '1821.92 -5.21',
 '3',
 'Argentina',
 '1773.88',
 '1770.65 3.23',
 '4',
 'France',
 '1759.78',
 '1764.85 -5.07',
 '5',
 'England',
 '1728.47',
 '1737.46 -8.99',
 '6',
 'Italy',
 '1726.14',
 '1713.86 12.28',
 '7',
 'Spain',
 '1715.22',
 '1716.93 -1.71',
 '8',
 'Netherlands',
 '1694.51',
 '1679.41 15.10',
 '9',
 'Portugal',
 '1676.56',
 '1678.65 -2.09',
 '10',
 'Denmark',
 '1666.57',
 '1665.47 1.10',
 '11',
 'Germany',
 '1650.21',
 '1658.96 -8.75',
 '12',
 'Croatia',
 '1645.64',
 '1632.15 13.49',
 '13',
 'Mexico',
 '1644.89',
 '1649.57 -4.68',
 '14',
 'Uruguay',
 '1638.71',
 '1640.95 -2.24',
 '15',
 'Switzerland',
 '1635.92',
 '1621.43 14.49',
 '16',
 'USA',
 '1627.48',
 '1635.01 -7.53',
 '17',
 'Colombia',
 '1611.04',
 '1604.07 6.97',
 '18',
 'Senegal',
 '1584.38',
 '1584.59 -0.21',
 '19',
 'Wales',
 '1569.82',
 '1582.13 -12.31',
 '20',
 

In [63]:
columns = fifa_ratings[:3]

rank = fifa_ratings[6::4]
teams = fifa_ratings[7::4]
pts = fifa_ratings[8::4]

In [64]:
df = pd.DataFrame(list(zip(rank, teams, pts)), columns=columns)

In [65]:
df

,RK,Team,Total Points
0,1,Brazil,1841.3
1,2,Belgium,1816.71
2,3,Argentina,1773.88
3,4,France,1759.78
4,5,England,1728.47
...,...,...,...
95,96,Vietnam,1226.65
96,97,Belarus,1221.06
97,98,Equatorial Guinea,1215.66
98,99,Lebanon,1211.71


In [66]:
df.loc[df['Team'] == 'IR Iran', 'Team'] = 'Iran'

## 3.2 Elo Wrangling

In [67]:
elo = soup(elo_table)

In [68]:
del elo[:4]

In [69]:
del elo[3:16]

In [70]:
columns_elo = elo[:3]

In [71]:
rank_elo = elo[3::16]
team_elo = elo[4::16]
rating_elo = elo[5::16]

In [72]:
df_elo = pd.DataFrame(list(zip(rank_elo, team_elo, rating_elo)), columns=columns)

In [73]:
df_elo.loc[df_elo['Team'] == 'South Korea', 'Team'] = 'Korea Republic'
df_elo.loc[df_elo['Team'] == 'United States', 'Team'] = 'USA'

## 3.3 Average Ratings Fifa / Elo

In [74]:
df['Total Points'] = df['Total Points'].astype('float64')

In [75]:
df_elo['Total Points'] = df_elo['Total Points'].astype('float64')

In [76]:
df_avg = df.merge(df_elo, how='left', on='Team')

In [77]:
df_avg2 = df_avg.drop('RK_y', axis=1)

In [78]:
df_avg2['avg_points'] = (df_avg2['Total Points_x'] + df_avg2['Total Points_y']) / 2

In [79]:
df_avg3 = df_avg2.drop(['Total Points_x', 'Total Points_y', 'RK_x'], axis=1).sort_values(by=['avg_points'], ascending=False).reset_index(drop=True)

In [80]:
df_avg4 = df_avg3.dropna()

In [81]:
df_avg4

,Team,avg_points
0,Brazil,2005.150
1,Argentina,1957.440
2,Belgium,1920.855
3,France,1882.390
4,Spain,1880.110
...,...,...
88,Benin,1319.735
89,Belarus,1317.530
90,Vietnam,1308.825
91,Lebanon,1274.355


In [85]:
teams_world_cup = ['Qatar', 'Ecuador', 'Senegal', 'Netherlands', 'England', 'Iran', 'USA', 'Wales', 'Argentina',
              'Saudi Arabia', 'Mexico', 'Poland', 'France', 'Australia', 'Denmark', 'Tunisia', 'Spain', 'Costa Rica',
              'Germany', 'Japan', 'Belgium', 'Canada', 'Morocco', 'Croatia', 'Brazil', 'Serbia', 'Switzerland',
              'Cameroon', 'Portugal', 'Ghana', 'Uruguay', 'Korea Republic']

In [86]:
df_teams_avg = df_avg4[df_avg4['Team'].isin(teams_world_cup)].reset_index(drop=True)
df_teams_avg['Team'] = df_teams_avg['Team'].str.lower()

In [87]:
df_teams_avg

,Team,avg_points
0,brazil,2005.150
1,argentina,1957.440
2,belgium,1920.855
3,france,1882.390
4,spain,1880.110
5,netherlands,1867.255
6,portugal,1840.280
7,england,1824.235
8,denmark,1818.785
9,germany,1805.105


In [88]:
df_teams_avg.to_csv('ratings_world_cup_2022.csv')

In [89]:
games_22 = pd.read_csv('world_cup_games_2022.csv')

In [90]:
games_22

,home_teams,away_teams
0,qatar,ecuador
1,senegal,netherlands
2,england,iran
3,usa,wales
4,france,australia
5,denmark,tunisia
6,mexico,poland
7,argentina,saudi arabia
8,belgium,canada
9,spain,costa rica


In [91]:
df_home = df_teams_avg.copy()
df_away = df_teams_avg.copy()

In [92]:
df_home = df_home.rename(columns={'Team': 'home_teams'})
df_away = df_away.rename(columns={'Team': 'away_teams'})

In [93]:
games_22 = games_22.merge(df_home, how='left', on='home_teams')
games_22 = games_22.merge(df_away, how='left', on='away_teams')

In [94]:
games_22

,home_teams,away_teams,avg_points_x,avg_points_y
0,qatar,ecuador,1559.945,1648.695
1,senegal,netherlands,1635.690,1867.255
2,england,iran,1824.235,1690.805
3,usa,wales,1712.740,1679.910
4,france,australia,1882.390,1603.860
5,denmark,tunisia,1818.785,1597.270
6,mexico,poland,1732.945,1678.795
7,argentina,saudi arabia,1957.440,1538.890
8,belgium,canada,1920.855,1620.000
9,spain,costa rica,1880.110,1623.295


In [95]:
prediction_games = games_22[['avg_points_x', 'avg_points_y']]

In [125]:
games_predicted = pd.read_csv('world_cup_predictions.csv')

In [126]:
games_predicted

,home_goals,away_goals
0,1,1
1,0,1
2,2,1
3,0,0
4,2,1
5,2,1
6,0,1
7,2,1
8,2,1
9,2,1


In [128]:
final_results = games_22.join([games_predicted])

In [129]:
final_results

,home_teams,away_teams,avg_points_x,avg_points_y,home_goals,away_goals
0,qatar,ecuador,1559.945,1648.695,1,1
1,senegal,netherlands,1635.690,1867.255,0,1
2,england,iran,1824.235,1690.805,2,1
3,usa,wales,1712.740,1679.910,0,0
4,france,australia,1882.390,1603.860,2,1
5,denmark,tunisia,1818.785,1597.270,2,1
6,mexico,poland,1732.945,1678.795,0,1
7,argentina,saudi arabia,1957.440,1538.890,2,1
8,belgium,canada,1920.855,1620.000,2,1
9,spain,costa rica,1880.110,1623.295,2,1
